In [1]:
import pandas as pd
import numpy as np
import os
import re

In [3]:
attendance = pd.DataFrame()
for year in range(18, 23):  # range is exclusive at the end
    filename = f"/Users/danielmatten/desktop/m/attendance_pub20{year}.sas7bdat"
    try:
        # Read SAS file
        df = pd.read_sas(filename)

        # Keep only relevant columns
        df.columns = df.columns.str.lower()

        # Drop missing values

        
        # Append to the main DataFrame
        attendance = pd.concat([attendance, df], ignore_index=True)
        print(f"{year} done")

    except FileNotFoundError:
        print(f"File {filename} not found. Skipping.")
    except KeyError:
        print(f"Required columns not found in {filename}. Skipping.")
        
for year in range(14,18):
    filename = f"/Users/danielmatten/desktop/m/pcaudit_pub20{year}.sas7bdat"
    try:
        # Read SAS file
        df = pd.read_sas(filename)

        # Keep only relevant columns
        df.columns = df.columns.str.lower()

        # Drop missing values
        columns_to_keep = ['mastid', 'grade', 'daysabs', 'daysmem']

        # Filter the DataFrame to keep only the specified columns
        df = df[columns_to_keep]
        
        # Append to the main DataFrame
        attendance = pd.concat([attendance, df], ignore_index=True)
        print(f"{year} done")

    except FileNotFoundError:
        print(f"File {filename} not found. Skipping.")
    except KeyError:
        print(f"Required columns not found in {filename}. Skipping.")
for year in range(10,14):
    filename = f"/Users/danielmatten/desktop/m/accdemopub20{year}.sas7bdat"
    try:
        # Read SAS file
        df = pd.read_sas(filename)

        # Keep only relevant columns
        df.columns = df.columns.str.lower()

        # Drop missing values

        columns_to_keep = ['mastid', 'grade', 'daysabs', 'daysmem']

        # Filter the DataFrame to keep only the specified columns
        df = df[columns_to_keep]
        # Append to the main DataFrame
        attendance = pd.concat([attendance, df], ignore_index=True)
        print(f"{year} done")

    except FileNotFoundError:
        print(f"File {filename} not found. Skipping.")
    except KeyError:
        print(f"Required columns not found in {filename}. Skipping.")


def clean_bytes(val):
    if isinstance(val, bytes):
        return str(val)[2:-1]  # str(b'xyz') => "b'xyz'" → "xyz"
    return val
for col in attendance.columns:
    attendance[col] = attendance[col].apply(clean_bytes)
    print(col)
    
#attendance['schl_yr'] = attendance['schl_yr'].str.split('-').str[1].astype(int)
#attendance.dropna(subset=['mastid'], inplace=True)
#attendance.to_csv("/Users/adamcartwright/NCERDC-MODEL/data/attendance_master.csv",index=False)

18 done
19 done
File /Users/danielmatten/desktop/m/attendance_pub2020.sas7bdat not found. Skipping.
21 done
22 done
14 done
15 done
16 done
17 done
10 done
11 done
12 done
13 done
schl_yr
dayp
daya
daye
lea
schlcode
mastid
grade
daysabs
daysmem


In [4]:
attendance.to_csv("../data/attendance_master.csv",index=False)

In [18]:
exit_ids = pd.read_csv('../data/exit_list.csv')
# If squeeze doesn’t work (Pandas 2.x), just:
# Then use to filter another DataFrame
filtered = attendance.merge(exit_ids,on='mastid',how='inner')
filtered = filtered[filtered['mastid'].isin(exit_ids['mastid'])]


In [19]:
filtered.drop(columns=['lea','schlcode','dayp'],inplace=True)

In [20]:
filtered['daya'] = filtered['daya'].fillna(filtered['daysabs'])

# Fill NaN values in 'daye' with the values from 'daysmem'
filtered['daye'] = filtered['daye'].fillna(filtered['daysmem'])

In [21]:
filtered.drop(columns=['daysmem','daysabs'],inplace=True)

In [22]:
filtered['grade'] = pd.to_numeric(filtered['grade'], errors='coerce')

# Step 2: Fill missing 'grade' values with the calculated formula
# Extract the year after the '-' in 'schlyear' and convert to numeric
filtered['schlyear_year'] = filtered['schl_yr'].str.split('-').str[0].astype(float)

# Calculate the missing grade values using the formula
filtered['grade'] = filtered['grade'].fillna(
    filtered['schlyear_year'] - filtered['effective_g9year'] + 9
)

In [23]:
filtered.drop(columns=['schl_yr','effective_g9year','schlyear_year'],inplace=True)

In [27]:
# Ensure daya and daye are numeric
filtered['daya'] = pd.to_numeric(filtered['daya'], errors='coerce')
filtered['daye'] = pd.to_numeric(filtered['daye'], errors='coerce')

# Compute absent percent
filtered['absent_pc'] = filtered['daya'] / filtered['daye']

# Filter to grades 6 through 9 only
grade_range = [3,4,5,6,7,8,9,10,11]
filtered = filtered[filtered['grade'].isin(grade_range)]

# Count how many distinct grades each student has in that range
grade_counts = filtered.groupby('mastid')['grade'].nunique()


# Group by mastid and grade and take mean absent_pc
agg_df = filtered.groupby(['mastid', 'grade'])['absent_pc'].mean().reset_index()

# Now pivot safely
pivot_df = agg_df.pivot(index='mastid', columns='grade', values='absent_pc')
pivot_df.columns = [f'absent_pc_grade_{g}' for g in pivot_df.columns]
pivot_df = pivot_df.reset_index()





In [28]:
# Strip '.0' from all column names
pivot_df.columns = [
    col.replace('.0', '') if isinstance(col, str) else col
    for col in pivot_df.columns
]


In [31]:
# Impute grade 7 if missing
pivot_df['absent_pc_grade_7'] = pivot_df['absent_pc_grade_7'].fillna(
    (pivot_df['absent_pc_grade_6'] + pivot_df['absent_pc_grade_8']) / 2
)

# Impute grade 8 if missing
pivot_df['absent_pc_grade_8'] = pivot_df['absent_pc_grade_8'].fillna(
    (pivot_df['absent_pc_grade_7'] + pivot_df['absent_pc_grade_9']) / 2
)


In [30]:
# Assuming 'pivot_df' is your DataFrame

# Count the number of NaN values per row
na_counts_per_row = pivot_df.isna().sum(axis=1)

# Create a summary table of the NaN counts per row
na_counts_per_row_summary = na_counts_per_row.value_counts().sort_index()

# Display the result
print(na_counts_per_row_summary)


0     69730
1    295742
2     68923
3     22065
4     15150
5     15909
6     22236
7      1683
8         9
Name: count, dtype: int64


In [31]:
import numpy as np

# Assuming 'pivot_df' is your DataFrame

# Step 1: Filter rows with 1 or fewer NaNs
filtered_df = pivot_df[pivot_df.isna().sum(axis=1) <= 1]

# Step 2: Impute missing values based on the conditions
def impute_missing_values(row):
    # If there is exactly one NaN in the row, find its position
    if row.isna().sum() == 1:
        # Check if NaN is in 'absent_pc_grade_3' or 'absent_pc_grade_11'
        nan_column = row[row.isna()].index[0]
        
        if nan_column not in ['absent_pc_grade_3', 'absent_pc_grade_11']:
            # Find the previous and next columns to impute from
            grade_number = int(nan_column.split('_')[-1])  # Get the grade number (e.g., 3 or 11)
            prev_column = f"absent_pc_grade_{grade_number - 1}"
            next_column = f"absent_pc_grade_{grade_number + 1}"

            # Check if both previous and next columns exist
            if prev_column in row.index and next_column in row.index:
                # Impute as the mean of the previous and next grade
                row[nan_column] = np.mean([row[prev_column], row[next_column]])

    return row

# Apply the imputation function to each row
filtered_df = filtered_df.apply(impute_missing_values, axis=1)

# Display the updated DataFrame (optional)


In [32]:
pivot_df = filtered_df.dropna()

In [33]:
pivot_df.to_csv("../data/attendance_mastid.csv",index=False)